# Media Operations Multi-agent Collaboration with Strands Agents SDK

In this notebook, we'll build a sophisticated multi-agent system for media operations using the Strands Agents SDK. This system demonstrates the "Agents as Tools" pattern, where specialized agents work together to analyze media content, ensure compliance with standards, and generate appropriate metadata for different platforms.

## What You'll Learn
- How to implement the "Agents as Tools" pattern using Strands Agents SDK
- How to create specialized agents with focused responsibilities
- How to build a hierarchical agent system with a supervisor agent
- How to leverage Amazon Bedrock Knowledge Bases for compliance checking
- How to implement quality control in an agent workflow

## The Concept: Agents as Tools

"Agents as Tools" is an architectural pattern in AI systems where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

- A primary "orchestrator" agent handles user interaction and determines which specialized agent to call
- Specialized "tool agents" perform domain-specific tasks when called by the orchestrator

This approach mimics human team dynamics, where a manager coordinates specialists, each bringing unique expertise to solve complex problems.

### Our Multi-agent Architecture

In this notebook, we'll implement a media operations workflow with the following specialized agents:

![Media Operations Agents](../../static/images/03-media-operations-agent.png)

1. **Synopsis Agent**: Generates metadata based on video analysis and requirements
2. **Compliance Analyzer**: Retrieves compliance requirements from a knowledge base
3. **QC Agent**: Performs quality checks on the generated metadata
4. **Supervisor Agent**: Orchestrates the entire workflow

<div class="alert-warning">
    <b>Warning:</b>
    <p>
    1) please make sure to run <b>00-prerequisites.ipynb</b> to properly setup all the packages.
    </p>
</div>

In [ ]:
from strands import Agent, tool
import boto3
import json_repair
import json
import sagemaker
from strands.models import BedrockModel
from IPython.display import JSON, display


boto_session = boto3.session.Session()
sess = sagemaker.Session(boto_session=boto_session)

# Initialize bedrock agent runtime
bedrock_agent_runtime = boto_session.client("bedrock-agent-runtime")

# Define knowledge base parameters
knowledge_base_name = "compliance-kb"
knowledge_base_description = "knowledge base contain guideline requirements and output schema for different type of video posts"
new_bucket = f"{knowledge_base_name}-{sess.boto_region_name}"
# Bedrock
bedrock_model = BedrockModel(
  model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
  temperature=0.1,
)


In [ ]:
%store -r video_analysis
%store -r additional_info

Load `video_analysis` parameter if it's not available from previous module

In [ ]:
try:
    display(JSON(video_analysis))
except Exception as e:

    with open('video_analysis.json', 'r') as file:
        video_analysis = json.load(file)
    display(JSON(video_analysis))

Define `additioanl_info` parameter if it is not available from previous module

In [ ]:
try:
    print(additional_info)
except Exception as e:

    additional_info = '## Film Identification\n\nThis clip is from the short film **"Meridian"** (2016). \n\n## Cast Information Detected\nThe video analysis identified three key actors in the clip:\n- Reid Scott as Detective Jake Sullivan\n- Kevin Kilner as Captain Mac Foster\n- Elyse Levesque as the mysterious woman\n\n## About the Film\n"Meridian" appears to be a mystery/thriller short film set in 1947 Los Angeles. The clip features Detective Jake Sullivan investigating a series of mysterious disappearances of three divorced men near El Matador Beach. The only connection between these men is their divorced status and minimal criminal records.\n\nThe plot revolves around these strange disappearances and a witness account of seeing one of the victims standing on a distinctive rock formation at the shoreline before vanishing during an unnaturally sudden storm. In his place appeared a mysterious woman in a white dress with "alabaster skin" who seemed otherworldly.\n\nThe clip suggests that Detective Sullivan himself may have gone missing while investigating, as we see Captain Foster arriving at the El Matador location looking for him at the end of the scene.\n\nThe story appears to blend elements of noir detective drama with supernatural overtones.'
    print(additional_info)

In [ ]:
import sys
sys.path.insert(0, '..')

from helper.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)

kb = KnowledgeBasesForAmazonBedrock()

## Building a Knowledge Base for Compliance Requirements

To enable our Compliance Analyzer agent to retrieve relevant compliance documents, we'll create an Amazon Bedrock Knowledge Base. This knowledge base will contain various compliance standards for different types of media content and platforms.

The knowledge base will store documents that define:
- Requirements for different types of content (Film, Sports)
- Format specifications for different platforms (Social Media, Website, Internal Logging)
- Output schemas that our metadata must follow

Let's create the knowledge base and populate it with our compliance documents:

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    new_bucket
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

In [ ]:
standards_folder = "standards"

!aws s3 sync {standards_folder} s3://{new_bucket}/

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

## Implementing Specialized Agents

### Compliance Analyzer

The Compliance Analyzer agent is responsible for retrieving and interpreting compliance requirements from a knowledge base. This agent:

1. Receives a query about the type of content being processed
2. Uses the Amazon Bedrock Knowledge Base to retrieve relevant compliance documents
3. Extracts and formats the requirements and output schema from these documents

This agent demonstrates how to integrate external knowledge sources into your agent workflow using the Strands Agents SDK.

In [ ]:
@tool
def compliance_analyzer(query: str) -> str:
    """Analyze compliance standards, and extract relevant insights from the docs"""
    print("📄 Compliance analyizer analyzes documentation...")

    @tool
    def retreive_docs(query: str, max_results=1) -> str:
        try:
            # Call the retrieve API
            response = bedrock_agent_runtime.retrieve(
                knowledgeBaseId=kb_id,
                retrievalQuery={
                    'text': query
                },
                retrievalConfiguration={
                    'vectorSearchConfiguration': {
                        'numberOfResults': max_results
                    }
                }
            )
            
            # Extract and return the retrieved results
            return json.dumps(response.get('retrievalResults', []))
        
        except Exception as e:
            print(f"Error retrieving from knowledge base: {e}")
            return ""

        
    # System prompt for generating answers from retrieved information
    ANSWER_SYSTEM_PROMPT = """
    You are a compliance documentation analyzer that identifies and analyzes relevant compliance document base on 
    the task. Upon receiving a task, you will locate the most appropriate compliance documents, summarize 
    the key compliance requirements, and extract the corresponding JSON structure as specified in the documentation. 
    You will output the JSON object ONLY. The JSON only has a "requriements" attribute is a list of mandatory 
    requirements and a "schema" filed, which is the complete JSON example.
    """

    agent = Agent(model=bedrock_model,
                  tools=[retreive_docs],
                  system_prompt=ANSWER_SYSTEM_PROMPT)

    return  agent(query)


In [ ]:
output = compliance_analyzer("preparing this film video for social media post")
json_output = json_repair.loads(output.message['content'][0]['text'])

requirements = json_output['requirements']
print(f"Requirements:\n\n{str(requirements)}\n")
output_schema = json_output['schema']
print(f"Output Schema:\n\n{str(output_schema)}\n")

### Synopsis Agent

The Synopsis Agent is responsible for generating and enhancing metadata for video content. This agent:

1. Creates initial metadata based on video analysis, additional information, and compliance requirements
2. Can enhance existing metadata based on feedback from the QC Agent
3. Ensures the output follows the required format and includes all mandatory information

This agent demonstrates how to implement specialized tools within an agent to handle different aspects of a task (creation vs. enhancement).

In [ ]:
@tool
def synopsis_agent(query: str) -> str:
    """generate and enhace a factually correct and compliant video metadata based on the requirements, feedback, and expected output format."""
    print("✨ Synopsis agent creating ...")

    @tool
    def create_metadata(video_analysis: str, additional_info: str, requirements: str, output: str) -> str:
        
        AGENT_SYSTEM_PROMPT = """
        You are a media operations specialist responsible for analyzing raw analysis of a video and additional
        information about the video, the produce a compliant metadata following the requirements and output format.
        """
        agent = Agent(model=bedrock_model, system_prompt=AGENT_SYSTEM_PROMPT)

        prompt = """
            Here is the video analysis: 
            {video_analysis}
            
            Here is additional info about the video: 
            {additional_info}

            Here is the requirements:
            {requirements}

            Please generate and output the metadata in following output format ONLY.
            {output}
        """.format(video_analysis=video_analysis, additional_info=additional_info, requirements=requirements, output=output)

        return agent(prompt=prompt)
    
    @tool
    def enhance_metadata(current_metadata: str, feedback: str, video_analysis:str, additional_info: str, requirements: str, output: str) -> str:
        AGENT_SYSTEM_PROMPT = """
         You are a media operations specialist responsible for enhancing existing video metadata base on feedback. your are provided
         with the original video analysis for reference, as well as the requirements and output format to follow. Finally output needs
         to account for all the provided information.
         """
         
        agent = Agent(model=bedrock_model, system_prompt=AGENT_SYSTEM_PROMPT)

        prompt = """
            Here is the current metadata: {current_metadata}
            Here is the feedback: {feedback}
            Here is the video analysis: {video_analysis}
            Here is additional info about the video: {additional_info}
            Here is the requirements: {requirements}
            Please generate and output the metadata in following output format ONLY.
            {output}
        """.format(current_metadata=current_metadata, feedback=feedback, video_analysis=video_analysis, additional_info=additional_info, requirements=requirements, output=output)

        return agent(prompt=prompt)
    
    # System prompt for generating answers from retrieved information
    ANSWER_SYSTEM_PROMPT = """
    You are a media operations specialist, Your task is to generate or enhace a factually correct and compliant video metadata based on provided information.
    
    Core behavior:
    1. You should always create the metadata based on available information, do not manufacture facts on your own. 
    2. You should always provide clear, direct answers and present information in a easy-to-understand manner.
    3. Do not ask user for additioanl detail if you can not find it in the available information.
    4. Output video metadata ONLY, skip explanation.
    """

    agent = Agent(model=bedrock_model,
                  tools=[create_metadata, enhance_metadata],
                  system_prompt=ANSWER_SYSTEM_PROMPT)

    return agent(query)

In [ ]:
test_query = f"""
Here is the raw video analysis:
{str(video_analysis)}

Here is additional information found:
{str(additional_info)}

Here is the requirements: 
{str(requirements)}

Here is the output schema format:
{str(output_schema)}
"""

output = synopsis_agent(test_query)

### QC Agent

The Quality Control (QC) Agent is responsible for verifying that the generated metadata meets all compliance requirements. This agent:

1. Receives the generated metadata, requirements, and output schema
2. Checks for any violations of hard requirements
3. Returns feedback if violations are found or "Pass" if the metadata is compliant

This agent demonstrates how to implement validation logic within the multi-agent workflow.

In [ ]:
@tool
def qc_agent(query: str) -> str:
    """QC check video metadata against the requirements and expected output schema."""
    print("✅ QC agent reviewing...")

    @tool
    def word_count(text: str) -> int:
        """Count words in text.

        This docstring is used by the LLM to understand the tool's purpose.
        """      
        return len(text.split())
    
    # System prompt for generating answers from retrieved information
    ANSWER_SYSTEM_PROMPT = """
    You are a Quality Check (QC) Agent, You task is to conduct a thorough review of the video
    metadata against requirements and expected output format. Report hard requirement violations as feedback.
    
    Core behavior:
    1. You should always review base on available information, do not manufacture facts on your own. 
    2. You should always provide clear, direct answers and present information in a easy-to-understand manner.
    3. Do not ask user for additioanl detail if you can not find it in the available information.
    4. Only report hard requirement violations as feedback. If None, just say "Pass". Skip any explanation.
    """

    agent = Agent(model=bedrock_model,
                  tools=[word_count],
                  system_prompt=ANSWER_SYSTEM_PROMPT)

    return agent(query)

In [ ]:
video_metadata = {
    "description": "👻 Something spooky is happening at El Matador beach! Watch @KevinKilner as Captain Foster investigate mysterious disappearances in 'Meridian' (2016), featuring @ReidScott as Detective Sullivan and @ELevesque as the enigmatic woman in white. Don't miss this supernatural thriller! #Meridian #Supernatural"
}

test_query = f"""
Here is the metadata:
{str(video_metadata)}

Here is the requirements:
{str(requirements)}
 
Here is the output format:
{str(output_schema)}
"""

qc_agent(test_query)

### Supervisor Agent

The Supervisor Agent is the orchestrator that coordinates the entire workflow. This agent:

1. Analyzes the raw video analysis and additional information to determine the content type
2. Uses the Compliance Analyzer to retrieve relevant requirements and output format
3. Instructs the Synopsis Agent to generate metadata based on these requirements
4. Sends the generated metadata to the QC Agent for validation
5. If issues are found, sends the metadata back to the Synopsis Agent with feedback for enhancement
6. Repeats steps 3-5 until the metadata passes QC checks
7. Returns the final compliant metadata

This agent demonstrates the power of the "Agents as Tools" pattern, where a high-level agent orchestrates specialized agents to complete a complex workflow.

In [ ]:
# Level 1 - Executive Coordinator
COORDINATOR_SYSTEM_PROMPT = """
You are an executive coordinator who oversees complex analyses across multiple domains.
For compliance questions, use compliance_analyzer tool.
For metadata generataion and enhancement, use synopsis_agent tool.
For quality check, use qc_agent tool.

You process should be:
1. analyze the raw video analysis and additional info to determine what type of video (e.g.: Film or Sports)
2. get the compliance requirements and output format base on the user define task
3. generate or enhacement video metadata base on the requirements and ouptut format.
4. repeat step 3 until you pass the quality check.
5. return the final metadata in the compliant JSON output format. Skip any explanation
"""

In [ ]:
# Create the coordinator agent with all tools
supervisor = Agent(
    model=bedrock_model,
    system_prompt=COORDINATOR_SYSTEM_PROMPT,
    tools=[compliance_analyzer, synopsis_agent, qc_agent],
    callback_handler=None
)


In [ ]:
test_input=f"""
The task is to create a compliant social media post for the following video

Here is the raw video analysis:
{video_analysis}

Here is additional information found:
{additional_info}
"""
output = supervisor(test_input)
print("\nFinal output:\n")
json_output = json_repair.loads(output.message['content'][0]['text'])
display(JSON(json_output))

## Clean Up Resources

After completing the notebook, it's important to clean up the resources we've created to avoid unnecessary charges. The following cell will delete the knowledge base we created for this demonstration.

**Note**: Always make sure to clean up resources when you're done with them, especially in a cloud environment.

In [ ]:
# kb.delete_kb(knowledge_base_name)

## Conclusion

In this notebook, we've demonstrated how to build a sophisticated multi-agent system for media operations using the Strands Agents SDK. We've implemented the "Agents as Tools" pattern to create a hierarchical system where specialized agents work together to accomplish complex tasks.

### Key Takeaways

1. **Specialized Agents**: We created focused agents for compliance analysis, metadata generation, and quality control
2. **Hierarchical Orchestration**: Our supervisor agent coordinated the workflow between specialized agents
3. **Knowledge Integration**: We leveraged Amazon Bedrock Knowledge Bases to provide compliance information to our agents
4. **Iterative Refinement**: Our system implemented a feedback loop where metadata is improved based on QC feedback

The Strands Agents SDK provides a powerful framework for building complex multi-agent systems that can tackle real-world problems through collaboration and specialization.